# For testing

```
python python_files/train_sft.py `
  --train-ids-npy final_npy/train_input_ids.npy `
  --train-mask-npy final_npy/train_loss_mask.npy `
  --valid-ids-npy final_npy/valid_input_ids.npy `
  --valid-mask-npy final_npy/valid_loss_mask.npy `
  --config configs/model_config_124M_sft.json `
  --base-ckpt pretrain_checkpoint/best_base.pt `
  --run-dir fine_tuning_small `
  --lora --lora-r 16 --lora-alpha 32 --lora-drop 0.05 `
  --lora-targets "attn.qkv,attn.out_proj,mlp.fc,mlp.proj" `
  --micro-bsz 8 --accum 8 --workers 0 --prefetch 2 `
  --amp bf16 `
  --quick-every 0 --eval-every 2000 --periodic-every 0 --log-every 1 `
  --ema 0.9999 --ema-trainable-only `
  --total-updates 2000
```

```
python python_files/train_sft.py `
  --train-ids-npy final_npy/train_input_ids.npy `
  --train-mask-npy final_npy/train_loss_mask.npy `
  --valid-ids-npy final_npy/valid_input_ids.npy `
  --valid-mask-npy final_npy/valid_loss_mask.npy `
  --config configs/model_config_124M_sft.json `
  --run-dir fine_tuning_small `
  --lora --lora-r 16 --lora-alpha 32 --lora-drop 0.05 `
  --lora-targets "attn.qkv,attn.out_proj,mlp.fc,mlp.proj" `
  --micro-bsz 8 --accum 8 --workers 0 --prefetch 2 `
  --amp bf16 `
  --quick-every 0 --eval-every 3500 --periodic-every 2000 --log-every 1 `
  --ema 0.9999 --ema-trainable-only `
  --total-updates 3500 `
  --resume-from latest.pt
```

## RESULT
SYSTEM SPEC: <br/>
OS: Windows 11 Home 64-bit <br/>
CPU: AMD Ryzen 9 5900HS (≈4GHz) <br/>
GPU: NVIDIA GeForce RTX 3080 Laptop GPU (VRAM 8GB) <br/>
RAM: 40GB <br/>

**RUNTIME**: 18m 54s

# (TL;DR)
there was a latancy every `--eval-eval` <br/>
formula:  <br/>
ceil(N_valid / batch_size) × forward_time_per_batch + small overhead <br/>
- batch_size = 8 (same as `--micro-bsz`)
- forward_time_per_batch = 80.6ms = 0.0806s (from `[profile] forward=80.6ms...`)
- N_valid ≈ 13,037

=> ceil(13037/8) ≈ 1635 <br/>
=> 1635 x 0.0806s ≈ 132s ≈ 2m <br/>
=> 2m + small overhead <br/>
yeah. in fact, it took around 3 minutes. so, i guess that's expected


# Real Run

```
python python_files/train_sft.py `
  --run-dir fine_tuning `
  --base-ckpt pretrain_checkpoint/best_base.pt `
  --config configs/model_config_124M_sft.json `
  --train-ids-npy final_npy/train_input_ids.npy `
  --train-mask-npy final_npy/train_loss_mask.npy `
  --valid-ids-npy final_npy/valid_input_ids.npy `
  --valid-mask-npy final_npy/valid_loss_mask.npy `
  --lora --lora-r 32 --lora-alpha 64 --lora-drop 0.05 `
  --lora-targets "attn.qkv,attn.out_proj,mlp.fc,mlp.proj" `
  --micro-bsz 8 --accum 8 --workers 0 --prefetch 2 `
  --amp bf16 `
  --ema 0.9999 --ema-trainable-only --ema-every 4 `
  --base-lr 2e-4 --min-lr-ratio 0.02 --warmup-pct 0.03 `
  --label-smoothing 0.0 `
  --quick-every 0 --eval-every 1000 --log-every 500 --periodic-every 1000 --clip-every 100 `
  --total-updates 31000
  ```

**ok. I saw overfitting graph (valid_loss was increasing)** <br/>
-> this is fixed now

## RESULT
SYSTEM SPEC: <br/>
OS: Windows 11 Home 64-bit <br/>
CPU: AMD Ryzen 9 5900HS (≈4GHz) <br/>
GPU: NVIDIA GeForce RTX 3080 Laptop GPU (VRAM 8GB) <br/>
RAM: 40GB <br/>

**RUNTIME**: roughly 35 hours <br/>